<a href="https://colab.research.google.com/github/janaat22/NLP_NER/blob/master/NER_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [0]:
pip install sklearn-crfsuite


In [0]:
pip install nltk

In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
nltk.download('gazetteers')

[nltk_data] Downloading package gazetteers to /root/nltk_data...
[nltk_data]   Package gazetteers is already up-to-date!


True

In [0]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [0]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn_crfsuite import CRF, scorers, metrics
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

from nltk.stem.porter import *
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import names, ieer, gazetteers

In [0]:
locations = gazetteers.words()
locations = [x.lower() for x in locations]
names_list = names.words()
names_list = [x.lower() for x in names_list]

In [0]:
word = 'LA'
word.lower() in locations

True

In [0]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:


def file_to_list(file_name):  
  training_key_value = []
  word_dict = {}
  tags = {}
  word_list = []
  sentence = 1
  file1 = open(file_name, 'r')
  Lines = file1.readlines() 
  for line in Lines:    
    training_key_value = line.split()
    if(len(training_key_value) == 2):
      word = training_key_value[0]
      tag = training_key_value[1]
      word_dict[word] = tag
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      word_list.append(([sentence, word, tagged[0][1], tag])) 
      tags.update(tagged)
    else:
      sentence = sentence + 1
    file1.close()
  return word_dict,word_list

#Train data
word_dict_train, word_list_train = file_to_list('train.txt')
# print(word_dict_train.keys())
# array_of_words_train = np.array(list(word_dict_train.keys()))
# array_of_values_train = np.array(list(word_dict_train.values()))
# X_Y_train = np.column_stack((array_of_words_train, array_of_values_train))

#Dev data
word_dict_dev,word_list_dev = file_to_list('dev.txt')
# array_of_words_dev = np.array(list(word_dict_dev.keys()))
# array_of_values_dev = np.array(list(word_dict_dev.values()))
# X_Y_dev = np.column_stack((array_of_words_dev, array_of_values_dev))




In [0]:
def file_to_list_test(file_name):  
  training_key_value = []
  word_dict = {}  
  word_list = []
  training_key_value = [] 
  test_word_list = []
  file1 = open(file_name, 'r')
  sentence_num = 1
  Lines = file1.readlines() 
  # print("print" + str(len(Lines[11].split('\n'))))
  for line in Lines:    
    if(len(line.split()) > 0):      
      word = line.replace('\n', '')
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      # print(sentence_num)
      test_word_list.append(([sentence_num, word, tagged[0][1]]))
    else:
      # print("\nelse")
      sentence_num = sentence_num + 1
  file1.close()
  return test_word_list



#Test data
word_list_test = file_to_list_test('test_no_tag.txt')


In [0]:
train_data = pd.DataFrame(word_list_train)
print(train_data.head(n = 4))

dev_data = pd.DataFrame(word_list_dev)
print(dev_data.head(n = 4))


test_data = pd.DataFrame(word_list_test)
print(test_data.head(n = 4))
# print(word_list_test)

   0                1    2  3
0  1  @SammieLynnsMom   NN  O
1  1         @tg10781   NN  O
2  1             they  PRP  O
3  1             will   MD  O
   0       1   2  3
0  1    STOP  NN  O
1  1    WHAT  WP  O
2  1  YOU'RE  NN  O
3  1   DOING  NN  O
   0       1    2
0  1      We  PRP
1  1     're  VBP
2  1  slaves  NNS
3  1      to   TO


In [0]:
train_data.columns = train_data.iloc[0]
train_data = train_data[1:]
train_data.columns = ['Sentence','Word','POS','Tag']
train_data = train_data.reset_index(drop=True)
#train_data.head(n=5)

dev_data.columns = dev_data.iloc[0]
dev_data = dev_data[1:]
dev_data.columns = ['Sentence','Word','POS','Tag']
dev_data = dev_data.reset_index(drop=True)
#dev_data.head(n=5)

test_data.columns = test_data.iloc[0]
test_data = test_data[1:]
test_data.columns = ['Sentence','Word','POS']
test_data = test_data.reset_index(drop=True)
test_data.head(n= 10)


,Sentence,Word,POS
0,1,'re,VBP
1,1,slaves,NNS
2,1,to,TO
3,1,a,DT
4,1,world,NN
5,1,we,PRP
6,1,mastered,VBN
7,1,.,.
8,2,I,PRP
9,2,want,NN


In [0]:
test_data.shape

(42412, 3)

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

class getsentence_ForTest(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p) for w, p in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist())
                                                      ]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [0]:
train_getter = getsentence(train_data)
train_sentences = train_getter.sentences

dev_getter = getsentence(dev_data)
dev_sentences = dev_getter.sentences

test_getter = getsentence_ForTest(test_data)
test_sentences = test_getter.sentences

#This is how a sentence will look like. 
print(train_sentences[0])

[('@tg10781', 'NN', 'O'), ('they', 'PRP', 'O'), ('will', 'MD', 'O'), ('be', 'VB', 'O'), ('all', 'DT', 'O'), ('done', 'VBN', 'O'), ('by', 'IN', 'O'), ('Sunday', 'NNP', 'O'), ('trust', 'NN', 'O'), ('me', 'PRP', 'O'), ('*wink*', 'NN', 'O')]


In [0]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        # 'word.lower()': word.lower(),
        # 'word[-3:]': word[-3:],
        # 'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'inLocations': word.lower() in locations,
        'inNames' : word.lower() in names_list,
        'isTag' : word.find('@'),
        'hasCapitalLetter' : word[0].isupper(),
        'stem' : stemmer.stem(word),        
        # 'lemma' : lemmatizer.lemmatize(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            # '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:inLocations': word1.lower() in locations,
            '-1:inNames' : word1.lower() in names_list,
            '-1:hasCapitalLetter' : word1[0].isupper(),
            '-1:stem' : stemmer.stem(word1),            
            # '-1:lemma' : lemmatizer.lemmatize(word1),
             
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            # '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:inLocations': word1.lower() in locations,
            '+1:inNames' : word1.lower() in names_list,
            '+1:hasCapitalLetter' : word1[0].isupper(),
            '+1:stem' : stemmer.stem(word1),            
            # '+1:lemma' : lemmatizer.lemmatize(word),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
print(test_sentences[5])
sent2features(train_sentences[3])[0]
# sent2features(test_sentences[7])[0]

[('Happy', 'JJ'), ('v-day', 'NN'), ('😌', 'NN'), ('❤', 'NN'), ('️', 'NN'), ('😂😂😂😂', 'NN'), ('#noshame', '#')]


{'+1:hasCapitalLetter': False,
 '+1:inLocations': False,
 '+1:inNames': False,
 '+1:postag': 'RB',
 '+1:postag[:2]': 'RB',
 '+1:stem': 'perhap',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 'BOS': True,
 'bias': 1.0,
 'hasCapitalLetter': False,
 'inLocations': False,
 'inNames': False,
 'isTag': 0,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'stem': '@ls_n',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False}

In [0]:
#Creating the train and test set

X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_dev = [sent2features(s) for s in dev_sentences]
y_dev = [sent2labels(s) for s in dev_sentences]

X_test = [sent2features(s) for s in test_sentences]


In [0]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
crf = crf.fit(X_train,y_train)

In [0]:
pred = cross_val_predict(estimator=crf, X = X_train,y = y_train, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [0]:
#evaluate the model
report = flat_classification_report(y_pred=pred, y_true=y_train)
print(report)
# f1 = metrics.flat_f1_score(y_train, y_pred, average='weighted', labels=labels)
# print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company       0.88      0.43      0.58       171
     B-group       0.68      0.16      0.26       106
  B-location       0.77      0.54      0.63       380
     B-other       0.55      0.23      0.32       225
    B-person       0.78      0.63      0.70       449
   B-product       0.83      0.15      0.26        97
     B-title       0.59      0.19      0.29        68
   I-company       0.50      0.08      0.14        36
     I-group       0.60      0.14      0.23        84
  I-location       0.62      0.34      0.44       154
     I-other       0.51      0.29      0.37       320
    I-person       0.77      0.72      0.74       215
   I-product       0.74      0.17      0.28        80
     I-title       0.56      0.19      0.29        77
           O       0.97      1.00      0.98     44006

    accuracy                           0.96     46468
   macro avg       0.69      0.35      0.43     46468
weighted avg       0.96   

In [0]:
# labels = list(crf.classes_)

# labels.remove('O')
print(labels)


['B-location', 'I-location', 'B-title', 'I-title', 'B-company', 'B-product', 'B-person', 'B-other', 'I-other', 'B-group', 'I-group', 'I-product', 'I-company', 'I-person']


In [0]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

0.29913491761344696

In [0]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("Accuracy: " + str(accuracy))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))
print("Recall : " + str(recall))
recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.700     0.179     0.286        39
   I-company      0.000     0.000     0.000        10
     B-group      0.444     0.036     0.067       111
     I-group      0.222     0.042     0.070        48
  B-location      0.609     0.435     0.508       154
  I-location      0.569     0.358     0.439        81
     B-other      0.462     0.136     0.211       132
     I-other      0.198     0.206     0.202        97
    B-person      0.656     0.480     0.554       171
    I-person      0.609     0.589     0.599        95
   B-product      0.000     0.000     0.000        37
   I-product      0.000     0.000     0.000       121
     B-title      0.167     0.059     0.087        17
     I-title      0.111     0.067     0.083        15

   micro avg      0.507     0.254     0.339      1128
   macro avg      0.339     0.185     0.222      1128
weighted avg      0.427     0.254     0.299      1128

Accuracy: 0.9407134071340

# Hyper Parameter Optimization

In [0]:
Model_X = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 29.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=False, averaging=None,
                                 c=None, c1=0.1, c2=0.1,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_t...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fac9639e240>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighted, l

# Best Parameters

In [0]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.09894864874708563, 'c2': 0.008322715163805268}
best CV score: 0.4580033402173682
model size: 0.35M


In [0]:
crf = rs.best_estimator_
y_pred = crf.predict(X_dev)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("accuracy :" + str(accuracy))
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))
recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.615     0.205     0.308        39
   I-company      0.000     0.000     0.000        10
     B-group      0.375     0.027     0.050       111
     I-group      0.000     0.000     0.000        48
  B-location      0.589     0.429     0.496       154
  I-location      0.561     0.395     0.464        81
     B-other      0.429     0.159     0.232       132
     I-other      0.169     0.227     0.194        97
    B-person      0.629     0.515     0.566       171
    I-person      0.580     0.611     0.595        95
   B-product      0.667     0.054     0.100        37
   I-product      0.000     0.000     0.000       121
     B-title      0.250     0.118     0.160        17
     I-title      0.182     0.133     0.154        15

   micro avg      0.473     0.270     0.343      1128
   macro avg      0.360     0.205     0.237      1128
weighted avg      0.416     0.270     0.304      1128

accuracy :0.9394833948339

In [0]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-title -> I-title 8.119282
B-location -> I-location 7.999295
B-product -> I-product 7.899984
B-group -> I-group 7.839663
B-other -> I-other 7.575454
I-title -> I-title 7.370144
I-other -> I-other 7.103920
I-product -> I-product 7.074475
B-person -> I-person 6.957228
I-group -> I-group 6.727056
I-location -> I-location 6.245277
B-company -> I-company 6.187699
I-company -> I-company 5.520954
I-person -> I-person 4.107069
O      -> O       3.956403
O      -> B-person 2.702857
O      -> B-company 2.398870
O      -> B-group 2.116452
O      -> B-title 2.103919
O      -> B-product 2.053674

Top unlikely transitions:
B-location -> B-other 0.413640
I-location -> B-group 0.242053
B-person -> O       0.170685
I-company -> B-location 0.165424
I-person -> O       0.146537
B-company -> O       0.122398
B-company -> B-location 0.004443
I-location -> B-location -0.009856
I-other -> O       -0.011795
B-location -> O       -0.088287
I-title -> O       -0.090715
I-location -> O  

In [0]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
12.361380 B-company stem:twitter
11.444681 B-person stem:pope
10.371792 B-product stem:iphon
9.554440 B-company stem:facebook
9.345045 B-other  stem:xma
9.303597 B-person stem:4dbling
8.977755 B-product stem:ipod
8.847905 B-other  stem:ebgp
8.367108 B-company stem:youtub
8.345653 B-company stem:walmart
8.239830 B-other  stem:christma
8.138001 B-other  stem:dynamit
8.108571 B-other  stem:dem
8.108231 B-company stem:mcdonald
8.081373 B-company stem:wipro
7.919225 B-product +1:stem:devic
7.911177 B-group  stem:indigen
7.907480 B-location stem:lh
7.778233 B-person stem:sfitzy93
7.699673 B-location stem:jupit
7.676702 B-product stem:pringl
7.645453 B-location stem:mv
7.642943 B-company stem:yahoo
7.610326 B-person stem:o.d.b.
7.403901 B-title  stem:gigli
7.310880 B-person stem:jfk
7.296390 B-person stem:eminem
7.219710 B-person stem:madi
7.059827 O        stem:tuesday
7.036540 B-product stem:iphone4

Top negative:
-2.563158 O        +1:stem:song
-2.564631 O        stem:rose
-2

In [0]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(y_pred[0][0])

O


In [0]:
outF = open("result.txt", "w")
outR = open("test_no_tag.txt", "r")
lines = outR.readlines()

line_num = 0
for list in y_pred:
    for x in list:
      if len(lines[line_num])>0:        
        outF.write(lines[line_num].replace("\n","")+"\t\t\t" + x +"\n")
       
      line_num = line_num + 1
outF.close()